In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [5]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = None
train_size  = 45000
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
batch_size = 256
epochs = 200

In [6]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [7]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [8]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_valid = x_train_all[train_size:]
y_valid = y_train_all[train_size:]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [9]:
x_train, x_valid, x_test = x_train.reshape((-1, *image_shape)), x_valid.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [10]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(
    100000
).batch(
    batch_size
).prefetch(10)

In [11]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)

In [12]:
layers = tf.keras.layers

In [13]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', 
                  kernel_initializer=tf.keras.initializers.GlorotNormal())(img_input)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same',
                 kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotNormal())(x)
out = layers.Dense(10, kernel_initializer=tf.keras.initializers.GlorotNormal())(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [14]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    elif epoch < 90:
        return 1e-2
    elif epoch < 150:
        return 1e-3
    else:
        return 1e-4

In [15]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [17]:
model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

Epoch 1/200
176/176 [==============================] - 17s 98ms/step - loss: 2.0771 - accuracy: 0.2541 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/200
176/176 [==============================] - 14s 81ms/step - loss: 1.7951 - accuracy: 0.3717


KeyboardInterrupt: 

In [18]:
model.load_weights('./model_weights/simple_cnn_train=all-without-DA_ce.h5')

In [17]:
model.evaluate(x=x_test, y=y_test_all, verbose=0)

[3.936307430267334, 0.6393000483512878]

In [16]:
# model.save('./model_weights/simple_cnn_train=all-without-DA_ce.h5')

In [ ]:
# with pooling, acc can reach 72.2%
# w.o. pooling, acc is at most 67%

In [18]:
ce_loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

@tf.function
def targeted_fgsm(x, y_target, model, eps):
    with tf.GradientTape() as tp:
        tp.watch(x)
        y = model(x)
        loss = ce_loss(y_target, y)
    grad = tp.gradient(loss, x)
    return tf.clip_by_value(x - eps * tf.sign(grad), 0, 1)

@tf.function
def untargeted_fgsm(x, y_true, model, eps):
    with tf.GradientTape() as tp:
        tp.watch(x)
        y = model(x)
        loss = ce_loss(y_true, y)
    grad = tp.gradient(loss, x)
    return tf.clip_by_value(x + eps * tf.sign(grad), 0, 1)

In [20]:
tmp = untargeted_fgsm(x_test[:2048], y_test[:2048], model, 0.03)
model.evaluate(x_test, y_test)
model.evaluate(tmp[:2048], y_test[:2048])
onp.save('./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy', tmp.numpy())

 1/64 [..............................] - ETA: 0s - loss: 41.2614 - accuracy: 0.1250WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0020s vs `on_test_batch_end` time: 0.0039s). Check your callbacks.


64/64 [==============================] - 1s 11ms/step - loss: 40.9442 - accuracy: 0.0723


In [17]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128], verbose=0)

==========NTK============


[7.348448276519775, 0.40625]

In [18]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========CE============


[7.420608043670654, 0.4384765625]

In [19]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========MSE============


[6.9515461921691895, 0.451171875]

In [25]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128], verbose=0)

[5.4414167404174805, 0.546875]

In [26]:
file_list = ['batch_NTK_simple_decrease_variance.npy',
             'batch_NTK_simple_increase_variance.npy',
             'batch_NTK_CNN10_decrease_variance.npy',
             'batch_NTK_CNN10_increase_variance.npy']

for f in file_list:
    tmp = onp.load(f)
    print('evaluating file: %s'%(f))
    result = model.evaluate(tmp, y_test[:2048], verbose=0)
    print(result)

In [ ]:
tmp = onp.load('./batch_NTK_CNN10_decrease_variance_part_6.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

In [24]:
tmp = onp.load('./batch_NTK_simple.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[9.7962064743042, 0.34521484375]

In [16]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[5.862701416015625, 0.52490234375]

In [16]:
tmp = onp.load('./batch_NTK_cnn_19.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[7.934947967529297, 0.41943359375]